<a href="https://colab.research.google.com/github/deepjyotisaha85/eva4/blob/master/session15A/S15A_DenseDepth_Generation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/gdrive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/gdrive


<a href="https://colab.research.google.com/github/ialhashim/DenseDepth/blob/master/DenseDepth.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
!git clone https://github.com/deepjyotisaha85/DenseDepth.git

Cloning into 'DenseDepth'...
remote: Enumerating objects: 297, done.
remote: Counting objects: 100% (297/297), done.
remote: Compressing objects: 100% (173/173), done.
remote: Total 297 (delta 161), reused 231 (delta 119), pack-reused 0
Receiving objects: 100% (297/297), 11.80 MiB | 37.76 MiB/s, done.
Resolving deltas: 100% (161/161), done.


In [3]:
!wget https://s3-eu-west-1.amazonaws.com/densedepth/nyu.h5 -O ./DenseDepth/nyu.h5

--2020-05-10 15:18:54--  https://s3-eu-west-1.amazonaws.com/densedepth/nyu.h5
Resolving s3-eu-west-1.amazonaws.com (s3-eu-west-1.amazonaws.com)... 52.218.84.202
Connecting to s3-eu-west-1.amazonaws.com (s3-eu-west-1.amazonaws.com)|52.218.84.202|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 172897376 (165M) [application/h5]
Saving to: ‘./DenseDepth/nyu.h5’

./DenseDepth/nyu.h5 100%[===================>] 164.89M  21.5MB/s    in 8.7s    

2020-05-10 15:19:03 (19.0 MB/s) - ‘./DenseDepth/nyu.h5’ saved [172897376/172897376]



In [0]:
#Manually make a copy of fg_bg.zip as depth_fg_bg.zip
#cp '/content/gdrive/My Drive/Colab Notebooks/EVA4/S15A/output/fg_bg.zip' '/content/depth_fg_bg.zip'

In [4]:
!ls '/content/gdrive/My Drive/Colab Notebooks/EVA4/S15A/output/'

depth_27001.zip  depth_28501.zip  depth_2.zip	   fg_bg.zip
depth_27501.zip  depth_29001.zip  depth_30001.zip
depth_28001.zip  depth_29501.zip  depth_fg_bg.zip


In [0]:
import zipfile

archive = zipfile.ZipFile('/content/gdrive/My Drive/Colab Notebooks/EVA4/S15A/output/fg_bg.zip')
for file in archive.namelist():
    if file.startswith('image/'):
        archive.extract(file, '/content/')

In [13]:
!ls '/content/image' | wc -l

400000


In [0]:
!mkdir '/content/depthmap'

mkdir: cannot create directory ‘/content/depthmap’: File exists


In [0]:
!ls '/content/depthmap' | wc -l

27000


In [0]:
#!rm -rf '/content/depthmap'

In [7]:
!pwd

/content


In [8]:
cd DenseDepth

/content/DenseDepth


In [0]:
import sys
workingdir = '/content/DenseDepth'
sys.path.append(workingdir)

In [10]:
!ls

augment.py	demo.py		  examples   model.py	Tensorflow
callbacks.py	demo_rgb.npy	  layers.py  nyu.h5	test.py
data.py		DenseDepth.ipynb  LICENSE    PyTorch	train.py
demo_depth.npy	evaluate.py	  loss.py    README.md	utils.py


In [0]:
import numpy as np
from PIL import Image
import matplotlib.pyplot as plt
from tqdm import notebook
import zipfile

def load_images(path, start, end):
    loaded_images = []
    for i in range(start, end+1):
        x = np.clip(np.asarray(Image.open(f"{path}fg_bg_{str(i)}.jpg" ).resize((256, 256)), dtype=float) / 255, 0, 1)
        #x = np.clip(np.asarray(Image.open(f"{path}fg_bg_{str(i)}.jpg" ), dtype=float) / 255, 0, 1)
        loaded_images.append(x)
    return np.stack(loaded_images, axis=0)


def write_images(out_zip, path, start, outputs, inputs=None, gt=None, is_colormap=True, is_rescale=True, log=False):
    import matplotlib.pyplot as plt
    import skimage
    from skimage.transform import resize

    plasma = plt.get_cmap('gray')

    shape = (outputs[0].shape[0], outputs[0].shape[1], 3)
    if log:
        print(shape)
    
    #all_images = []
    start = start
    for i in notebook.tqdm(range(outputs.shape[0]),desc = f"Loading from {start} images") :
    
        if is_colormap:
            rescaled = outputs[i][:,:,0]
            if is_rescale:
                rescaled = rescaled - np.min(rescaled)
                rescaled = rescaled / np.max(rescaled)
  
            plt.figure(figsize=(2.24,2.24),dpi=100)
            matplotlib_image = plt.imshow(plasma(rescaled)[:,:,:3])
            pil_image = Image.fromarray(np.uint8( ( matplotlib_image.get_array()*255))).convert("L").resize((220, 220))
            #pil_image = Image.fromarray(np.uint8( ( matplotlib_image.get_array()*255))).convert("L")
            pil_image.save('imgtemp.jpg', optimize=True, quality=30)
            out_zip.write('imgtemp.jpg',f'depthmap/depth_{str(start)}.jpg')
            #if log:
            #    print("Saving Image: ", f"{path}depth_{str(start)}.jpg")
            #pil_image.save(f"{path}depth_{str(start)}.jpg")
            plt.close()
            pil_image = None 
            matplotlib_image = None
            start+=1

In [0]:
#!ls '/content/image'

In [0]:
import test
import time
import gc


INPATH = "/content/image/"
OUTPATH = "/content/depthmap/"

start_time = time.time()

#total_images = 400000
#total_images = 4000
batch_size = 500
#num_iter = int(total_images/batch_size)
num_iter = 55
image_start = 56501
run = 3

#print("Generating Depth Images for:", total_images, "in batches of:", batch_size, "with ", num_iter, "iterations")

model = test.loadmodel()

outputs = []
inputs = []
log = False

for  i in range(1, num_iter+1):
  iter_start = time.time()
  out_zip =zipfile.ZipFile(f'/content/gdrive/My Drive/Colab Notebooks/EVA4/S15A/output/depth_{str(run)}.zip', mode='a', compression=zipfile.ZIP_STORED)
  start = image_start
  end = image_start + batch_size - 1
  print("Generating Image: ", image_start, " to ", end)
  inputs = load_images(INPATH, start, end)
  print('\nLoaded ({0}) images of size {1}.'.format(inputs.shape[0], inputs.shape[1:]))
  outputs = test.predict_output(model, inputs, log=True)
  write_images(out_zip, OUTPATH, start, outputs.copy(), log=True)
  print("Iteration:", i, "--- %s seconds ---" % (time.time() - iter_start), " --- Total : %s seconds ---" % (time.time() - start_time))
  image_start = image_start + batch_size
  inputs = None
  outputs = None
  out_zip.close()
  gc.collect()

print("--- %s seconds ---" % (time.time() - start_time))


Loading model...
Generating Image:  56501  to  57000

Loaded (500) images of size (256, 256, 3).
(128, 128, 3)


/usr/lib/python3.6/zipfile.py:1378: UserWarning: Duplicate name: 'depthmap/depth_56501.jpg'
  return self._open_to_write(zinfo, force_zip64=force_zip64)



Iteration: 1 --- 49.029075622558594 seconds ---  --- Total : 60.15864562988281 seconds ---
Generating Image:  57001  to  57500

Loaded (500) images of size (256, 256, 3).
(128, 128, 3)



Iteration: 2 --- 42.28050518035889 seconds ---  --- Total : 103.408926486969 seconds ---
Generating Image:  57501  to  58000

Loaded (500) images of size (256, 256, 3).
(128, 128, 3)



Iteration: 3 --- 39.986884355545044 seconds ---  --- Total : 144.73867797851562 seconds ---
Generating Image:  58001  to  58500

Loaded (500) images of size (256, 256, 3).
(128, 128, 3)


...


Iteration: 48 --- 39.34311842918396 seconds ---  --- Total : 2388.753900527954 seconds ---
Generating Image:  80501  to  81000

Loaded (500) images of size (256, 256, 3).
(128, 128, 3)



Iteration: 49 --- 39.26445198059082 seconds ---  --- Total : 2446.6511766910553 seconds ---
Generating Image:  81001  to  81500

Loaded (500) images of size (256, 256, 3).
(128, 128, 3)



Iteration: 50 --- 39.499139070510864 seconds ---  --- Total : 2505.2592277526855 seconds ---
Generating Image:  81501  to  82000

Loaded (500) images of size (256, 256, 3).
(128, 128, 3)



Iteration: 51 --- 40.961061000823975 seconds ---  --- Total : 2565.56227350235 seconds ---
Generating Image:  82001  to  82500

Loaded (500) images of size (256, 256, 3).
(128, 128, 3)



Iteration: 52 --- 39.327467918395996 seconds ---  --- Total : 2624.8011379241943 seconds ---
Generating Image:  82501  to  83000

Loaded (500) images of size (256, 256, 3).
(128, 128, 3)
